# Analysis file for Ariane quantitative runs
gets data from .nc file into form similar to what's summarized in the stats.txt file, but usable in python <br>
based on code from https://nbviewer.jupyter.org/github/SalishSeaCast/analysis-susan/blob/master/notebooks/Ariane/Transport.ipynb

In [1]:
#import required libraries
import arrow
import datetime
import matplotlib.pyplot as plt
import netCDF4 as nc
import numpy as np
import os
import pandas as pd
import pickle
import xarray as xr

%matplotlib inline

In [2]:
mesh = nc.Dataset('/home/sallen/MEOPAR/grid/mesh_mask201702.nc')
gdepw = mesh.variables['gdepw_1d'][0]
bathy = nc.Dataset('/home/sallen/MEOPAR/grid/bathymetry_201702.nc')
lats = bathy.variables['nav_lat'][:]
lons = bathy.variables['nav_lon'][:]
mesh.close()
bathy.close()

## Functions

In [3]:
def calculate_all(section, allv):
    final = {}
    mean = {}    
    filename = 'ariane_positions_quantitative.nc'
    fullfile = os.path.join('/ocean/rbeutel/MOAD/analysis-becca/Ariane/JDF_salish/14sep17', filename)
    try:
        if os.stat(fullfile).st_size > 0:
            filegood = True
            data = xr.open_dataset(fullfile)
            final_section = data.variables['final_section'][:]
            final_transport = data.variables['final_transp'][:]
            transport = np.sum(np.where(final_section == section, final_transport, 0)) 
            #transport= the sum of particles through this section (m3/s), same as stats.txt summary once divided by 24
            for variable in allv:
                #this loop calculates the mean values shown in stats.txt final state tables
                if variable == 'final_age':
                    scale = 86400. # convert from seconds to days
                else:
                    scale = 1.
                final[variable] = data.variables[variable][:]/scale
                mean[variable] = np.sum(np.where(final_section == section, final_transport*final[variable], 0))/transport
        else:
            filegood = False
            transport = float('NaN')
            print ("empty file")
    except OSError:
        filegood = False
        transport = float('NaN')
        print ('Missing Data')
    return filegood, transport/24., mean

In [4]:
def get_data_all(start, endtime, section, allv): 
    #pretty sure the only difference between this and calculate_all is that this one seperates into days (susan wants me to do for whole month)
    timerange = arrow.Arrow.range('day', start, endtime)
    length = (endtime-start).days + 1
    transport = np.zeros(length)
    mean = {}
    for variable in allv:
        mean[variable] = np.zeros(length)
    time = []
    for i, r in enumerate(timerange):
        dir1 = r.format('DDMMMYY').lower()
#        print (dir1)
        filegood, transport[i], meanday = calculate_all(section, allv)
        for variable in allv:
            if filegood:
                mean[variable][i] = meanday[variable]
            else:
                mean[variable][i] = float('NaN')
        time.append(r.datetime)
    return time, transport, mean

In [5]:
section = 4 #care about number of section not whatever you named it, this is SoG1
allv = ['final_age', 'final_depth', 'final_salt', 'final_temp', 'final_lon', 
        'init_depth', 'init_salt', 'init_temp', 'init_lat']

In [6]:
start = datetime.datetime(2017, 9, 14) #first full day
endtime = datetime.datetime(2017, 9, 14) #last full day

In [7]:
get_data_all(start, endtime, section, allv)

([datetime.datetime(2017, 9, 14, 0, 0, tzinfo=tzutc())],
 array([35187.92129251]),
 {'final_age': array([3.32426375e+09]),
  'final_depth': array([-21.00670309]),
  'final_salt': array([30.60636982]),
  'final_temp': array([11.18815158]),
  'final_lon': array([-122.74932382]),
  'init_depth': array([-58.80713027]),
  'init_salt': array([32.736253]),
  'init_temp': array([8.71515225]),
  'init_lat': array([48.4158211])})